# Week 10 Assignment

Because I was unable to conduct our workshop this week, I'm keeping the assignment light as well.  Below you'll find just two steps for this week: one programming exercise and then a planning activity for your final project.

For clarification, the "final project" I've been referring to is your "final."  It is not a project in addition to a final exam.  They're one-in-the-same.

Please do the programming exercise and verify that your code works using the tests, then think about your final project and fill out the questions in the second part.

---
---

### 47.1: Filtering and summarizing data

For this work, you'll find a data file in `/data/complications_all.csv`.

Read in the data file and create a variable called `mo_hospitals` that contains a data frame from the `complications_all.csv` file, filtered down to only contain those hospitals from the state of Missouri (MO).

Then aggregate that data by hospital into a variable named `mo_summary`.  There are some key fields that we want to summarize:
* We want to know the earliest date that each hospital was participating in any program
* We want to know the latest date that each hospital stopped participating in any program
* We want to know the total number of patients in the denominators of these programs

Some things to note:
* You will need to convert the `Start Date` and `End Date` to actual datetime fields
* You will need to clean up and convert the `Denominator` field to just be numeric - the rule that you should use it to simply remove any records where the `Denominator` is `'Not Available'`


The final result of this step should be a new data frame called `mo_summary` that contains one row for each hospital and contains the min start date, max end date, and total denominator.  Use the names `start_date`, `end_date`, and `number` for those columns in `mo_summary`.


You do not need to create your code in the form of a function, just make sure your variable names match what I've described above so the tests work.

In [1]:
import pandas as pd
# This is just to show you the name to use for the variable you need to create for this step to pass.
data = pd.read_csv('/data/complications_all.csv')

In [2]:
filter = data['State'] == 'MO'

In [3]:
mo_hospitals = data[filter]

In [4]:
# These assertions will help make sure that you're on the right track.
assert(mo_hospitals['State'].unique() == ['MO'])
assert(mo_hospitals.shape == (2133,18))

In [5]:
start_date_str = mo_hospitals['Start Date'].astype(str)

In [6]:
start_date = pd.to_datetime(start_date_str, format = '%m/%d/%Y')

In [7]:
end_date_str = mo_hospitals['End Date'].astype(str)

In [8]:
end_date = pd.to_datetime(end_date_str, format = '%m/%d/%Y')

In [9]:
mo_hospitals['start_date'] = start_date

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
mo_hospitals['end_date'] = end_date

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
missing_denom = mo_hospitals['Denominator'] == 'Not Available'

In [12]:
mo_hospitals_denom = mo_hospitals[~missing_denom]

In [13]:
mo_summary_dates = mo_hospitals_denom.groupby('Facility Name')['start_date'].agg([('start_date', 'min'), ('end_date', 'max')])

In [14]:
denom_conv = mo_hospitals_denom['Denominator'].astype(int)

In [15]:
mo_hospitals_denom['number'] = denom_conv

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
mo_summary_denom = mo_hospitals_denom.groupby('Facility Name')['number'].agg([('number', 'sum')])

In [17]:
mo_summary_denom

,number
Facility Name,
BARNES JEWISH HOSPITAL,131313
BARNES-JEWISH ST PETERS HOSPITAL,15668
BARNES-JEWISH WEST COUNTY HOSPITAL,9622
BATES COUNTY MEMORIAL HOSPITAL,3117
BELTON REGIONAL MEDICAL CENTER,9270
...,...
TRUMAN MEDICAL CENTER LAKEWOOD,4297
UNIVERSITY OF MISSOURI HEALTH CARE,56493
WASHINGTON COUNTY MEMORIAL HOSPITAL,220


In [18]:
mo_summary = pd.merge(mo_summary_dates, mo_summary_denom, left_on = 'Facility Name', right_on = 'Facility Name', how = 'left')

In [19]:
mo_summary

,start_date,end_date,number
Facility Name,,,
BARNES JEWISH HOSPITAL,2015-04-01,2016-07-01,131313
BARNES-JEWISH ST PETERS HOSPITAL,2015-04-01,2016-07-01,15668
BARNES-JEWISH WEST COUNTY HOSPITAL,2015-04-01,2016-07-01,9622
BATES COUNTY MEMORIAL HOSPITAL,2015-07-01,2016-07-01,3117
BELTON REGIONAL MEDICAL CENTER,2015-04-01,2016-07-01,9270
...,...,...,...
TRUMAN MEDICAL CENTER LAKEWOOD,2015-04-01,2016-07-01,4297
UNIVERSITY OF MISSOURI HEALTH CARE,2015-04-01,2016-07-01,56493
WASHINGTON COUNTY MEMORIAL HOSPITAL,2015-07-01,2015-07-01,220


In [20]:
#For some reason the end date assertion is not quite coming out correctly. All the other assertions appear to be working though.

assert(mo_summary['number'].sum() == 1766908)
assert(mo_summary['start_date'].min() == pd.Timestamp(2015,4,1))
assert(mo_summary['end_date'].max() == pd.Timestamp(2018,6,30))
assert(mo_summary.shape == (108,3))
assert(mo_summary.loc['BARNES JEWISH HOSPITAL'].number == 131313)
assert(mo_summary.loc['BOONE HOSPITAL CENTER'].number == 63099)

AssertionError: 

---

### 47.2 Planning your final project

You should be thinking about the things we've been learning and how you can apply them to your final project.  Use the rubric to help guid your thinking and then answer the questions below.  This is meant as a guide to help you think through what you will do.

#### A) Data Access

Your project should include data from at least three distinct types of sources.  For example: AWS S3, Relational Databases, Internet, Web Services, local files.  List what data sources you're planning to use.

1. Local Files
2. Internet
3. Still working on this

#### B. Data Formats

Your project should include data that comes in different file formats.  For example: HL7, EDI, HTML, CSV, Excel, JSON, XML.  List what data formats you're planning to use.

CSV
Excel
HTML

#### C. Objective

What purpose would your project serve in a real work setting?  Take a couple of paragraphs to write down why this is an interesting product.

I don't quite have a fully formed idea yet for this, but I'll have it ready to go by class on Tuesday. I've found some interesting data from the Illinois DOT as they publish their crash records for the state. I want to look at contributing factors for crashes, and I have some ideas for other data to pull in for it (census data, number of liquor stores/bars in an area, etc). Still working to see what all is publicaly available. 

---

## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Save this note with Ctrl-S (or Cmd-S)
2. Skip down to the last command cell (the one starting with `%%bash`) and run that cell.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

In [ ]:
assert False, "DO NOT REMOVE THIS LINE"

---

In [21]:
%%bash
git pull
git add week10_assignment_2.ipynb
git commit -a -m "Submitting the week 10 programming assignment"
git push

Already up to date.
[main 442bc88] Submitting the week 10 programming assignment
 2 files changed, 686 insertions(+), 3 deletions(-)
 create mode 100644 week10/week10_assignment_2.ipynb


To github.com:nsokolis/hds5210-2021.git
   9de1f72..442bc88  main -> main



---

If the message above says something like _Submitting the week 8 programming assignment_ or _Everything is up to date_, then your work was submitted correctly.